## **Sales Forecasting of Retail Clothing Product Categories**

#### Input Parameters:

      ProductCategory-( Womens, Mens, Others)
      MonthlyNominalGDPIndexinMillion 
      MonthlyRealGDPIndexinMillion
      CPI 
      unemploymentrate
      CommercialBankInterestRateonCreditCardPlans
      FinanceRateonPersonalLoansatCommercialBanks24MonthLoan
      Earningsorwagesindollarsperhour
      CottonMonthlyPriceUScentsperPoundlbs
      Changein
      Averageuplandplantedmillionacres
      yieldperharvestedacre
      Millusein480lbnetwerightinmillionbales 
      Exports  
      SeaLevelPressavg 
      Visibilityavg
      Windavg 
      Precipsum
      Event 
      FederalHoliday


Importing required libraries

In [1]:
import os
import pandas as pd
import numpy as np
from sklearn import preprocessing
import joblib

Load Config

In [2]:
import yaml

#read yaml file
with open('../config/config.yaml') as file:
  config= yaml.safe_load(file)
  #print(config)

## Load Processed dataset

Get the Root Dir

In [3]:
#Get current working dir
cwd_path=os.getcwd()
#print(cwd_path)

#  Get the path of ROOT DIR
path_list = cwd_path.split(os.sep) #split path
ROOT_PATH=os.path.join(path_list[0],os.sep,*path_list[1:-2]) # To move two level up, remove last two str and join to create root path
#print(ROOT_PATH)

Read the clean & Merged data

In [4]:
# process Data file
Process_Data_file=os.path.join(ROOT_PATH,config['processed_data_dir'],config['processed_data_file_1'])
print(Process_Data_file)

# Read the Sales input file name
try:
    Data=pd.read_csv(Process_Data_file)
    #sales_data.head()
except FileNotFoundError:
    print("file {} does not exist".format(Process_Data_file))



C:\Users\Riyaz Mehendi\OneDrive - CriticalRiver Technologies Pvt. Ltd\Desktop\Retail_Sales_Forecast\data\processed\Merged_data.csv


In [5]:
Data=Data.sample(n =5)

In [6]:
Data.reset_index(drop=True, inplace=True)

In [7]:
Data

,ProductCategory,SalesInThousandDollars,MonthlyNominalGDPIndexinMillion,MonthlyRealGDPIndexinMillion,CPI,unemploymentrate,CommercialBankInterestRateonCreditCardPlans,FinanceRateonPersonalLoansatCommercialBanks24MonthLoan,Earningsorwagesindollarsperhour,CottonMonthlyPriceUScentsperPoundlbs,...,Exports,Tempavg,DewPointavg,Humidityavg,SeaLevelPressavg,Visibilityavg,Windavg,Precipsum,Event,FederalHoliday
0,WomenClothing,2961.0,16955.608626,15705.972667,259.596,6.6,11.85,10.22,24.35,90.96,...,9.75,-1.238095,-9.476190,57.809524,1018.190476,13.047619,10.761905,2.710000,0,2
1,OtherClothing,921.0,14340.701639,14351.786822,235.067,8.7,12.97,11.05,22.22,51.50,...,11.65,5.967742,-2.903226,57.064516,1021.064516,14.258065,11.387097,1.434194,0,0
2,MenClothing,702.0,17109.777026,15804.519068,259.971,6.7,11.83,10.09,24.50,96.95,...,9.95,3.709677,-7.290323,48.451613,1015.709677,14.741935,11.612903,3.008065,0,0
3,OtherClothing,1039.0,15864.502339,15226.204678,249.322,8.3,12.36,10.36,23.56,101.11,...,10.35,2.964286,-4.714286,59.750000,1016.357143,14.071429,12.607143,2.758571,0,3
4,WomenClothing,2579.0,14596.605447,14595.393094,238.380,10.0,13.71,10.89,22.27,66.82,...,10.00,12.870968,7.032258,69.354839,1016.774194,14.064516,9.677419,4.572581,1,1


In [8]:
Data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5 entries, 0 to 4
Data columns (total 26 columns):
 #   Column                                                  Non-Null Count  Dtype  
---  ------                                                  --------------  -----  
 0   ProductCategory                                         5 non-null      object 
 1   SalesInThousandDollars                                  5 non-null      float64
 2   MonthlyNominalGDPIndexinMillion                         5 non-null      float64
 3   MonthlyRealGDPIndexinMillion                            5 non-null      float64
 4   CPI                                                     5 non-null      float64
 5   unemploymentrate                                        5 non-null      float64
 6   CommercialBankInterestRateonCreditCardPlans             5 non-null      float64
 7   FinanceRateonPersonalLoansatCommercialBanks24MonthLoan  5 non-null      float64
 8   Earningsorwagesindollarsperhour             

In [9]:
Data= Data.drop(['Averageuplandharvestedmillionacres','Productionin480lbnetwerightinmillionbales',
                                               'Tempavg','DewPointavg','Humidityavg','SalesInThousandDollars'],axis=1)

In [10]:
#Data= Data.drop(['Averageuplandharvestedmillionacres','Productionin480lbnetwerightinmillionbales',
#                                                'Tempavg','DewPointavg','Humidityavg'],axis=1)
#Data.to_csv('out.csv')

In [11]:
Data

,ProductCategory,MonthlyNominalGDPIndexinMillion,MonthlyRealGDPIndexinMillion,CPI,unemploymentrate,CommercialBankInterestRateonCreditCardPlans,FinanceRateonPersonalLoansatCommercialBanks24MonthLoan,Earningsorwagesindollarsperhour,CottonMonthlyPriceUScentsperPoundlbs,Changein,Averageuplandplantedmillionacres,yieldperharvestedacre,Millusein480lbnetwerightinmillionbales,Exports,SeaLevelPressavg,Visibilityavg,Windavg,Precipsum,Event,FederalHoliday
0,WomenClothing,16955.608626,15705.972667,259.596,6.6,11.85,10.22,24.35,90.96,3.97,10.206,807,3.580,9.75,1018.190476,13.047619,10.761905,2.710000,0,2
1,OtherClothing,14340.701639,14351.786822,235.067,8.7,12.97,11.05,22.22,51.50,-6.72,9.296,799,3.720,11.65,1021.064516,14.258065,11.387097,1.434194,0,0
2,MenClothing,17109.777026,15804.519068,259.971,6.7,11.83,10.09,24.50,96.95,3.08,10.206,807,3.580,9.95,1015.709677,14.741935,11.612903,3.008065,0,0
3,OtherClothing,15864.502339,15226.204678,249.322,8.3,12.36,10.36,23.56,101.11,5.93,14.426,754,3.575,10.35,1016.357143,14.071429,12.607143,2.758571,0,3
4,WomenClothing,14596.605447,14595.393094,238.380,10.0,13.71,10.89,22.27,66.82,4.29,8.989,799,3.370,10.00,1016.774194,14.064516,9.677419,4.572581,1,1


In [14]:
Data['Event'].unique()

array([0, 1], dtype=int64)

In [15]:
Data.shape

(5, 20)

In [16]:
Data.dtypes

ProductCategory                                            object
MonthlyNominalGDPIndexinMillion                           float64
MonthlyRealGDPIndexinMillion                              float64
CPI                                                       float64
unemploymentrate                                          float64
CommercialBankInterestRateonCreditCardPlans               float64
FinanceRateonPersonalLoansatCommercialBanks24MonthLoan    float64
Earningsorwagesindollarsperhour                           float64
CottonMonthlyPriceUScentsperPoundlbs                      float64
Changein                                                  float64
Averageuplandplantedmillionacres                          float64
yieldperharvestedacre                                       int64
Millusein480lbnetwerightinmillionbales                    float64
Exports                                                   float64
SeaLevelPressavg                                          float64
Visibility

#### Enter the Inputs

In [ ]:
index=int(input('Please select a row number from 0 to 4 to choose your input: ' ))

In [ ]:
Data=Data.loc[index]

In [ ]:
Data

In [ ]:
Data.dtypes

#### Processing user input

In [ ]:
Data = pd.DataFrame(Data).transpose()

In [ ]:
## converting text data into numerical
Data.replace({'MenClothing': 0, 'WomenClothing': 1,'OtherClothing':2},inplace=True)

In [ ]:
Data

## Load the model

In [ ]:
# Prediction model filename
prediction_model_file=os.path.join(ROOT_PATH,config['models_dir'],config['prediction_model'])
#print(prediction_model_file)

In [ ]:
## load the model
import pickle
prediction_model = pickle.load(open(prediction_model_file, 'rb'))

In [ ]:
## lets predict the labels for x data

y_pred=prediction_model.predict(Data)
y_pred

#### Output 

In [ ]:
print("The sales value for choosen category input is:", int(y_pred))